# 多层感知机实践：预测混凝土抗压强度

在这个实践中，我们将学习如何使用PyTorch实现多层感知机(MLP)模型。我们将以预测混凝土抗压强度为例，这是土木工程中的一个重要问题。

## 1. 导入必要的库

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

## 2. 准备数据

我们将使用一个包含多个影响因素的混凝土强度数据集。这个数据集包含以下特征:
- 水泥用量 (kg/m^3)
- 水用量 (kg/m^3)
- 粗骨料用量 (kg/m^3)
- 细骨料用量 (kg/m^3)
- 外加剂用量 (kg/m^3)
- 龄期 (天)

目标变量是28天抗压强度 (MPa)。

In [ ]:
# 生成模拟数据
np.random.seed(42)
n_samples = 1000

cement = np.random.uniform(200, 400, n_samples)
water = np.random.uniform(150, 200, n_samples)
coarse_aggregate = np.random.uniform(700, 1200, n_samples)
fine_aggregate = np.random.uniform(600, 1000, n_samples)
admixture = np.random.uniform(5, 15, n_samples)
age = np.random.randint(7, 91, n_samples)

# 模拟一个非线性关系
strength = (
    0.05 * cement
    - 0.1 * water
    + 0.01 * coarse_aggregate
    + 0.02 * fine_aggregate
    + 0.5 * admixture
    + 0.1 * np.log(age)
    + np.random.normal(0, 5, n_samples)
)

# 创建DataFrame
data = pd.DataFrame({
    '水泥': cement,
    '水': water,
    '粗骨料': coarse_aggregate,
    '细骨料': fine_aggregate,
    '外加剂': admixture,
    '龄期': age,
    '强度': strength
})

print(data.head())
print("\n数据集形状:", data.shape)

## 3. 数据预处理

In [ ]:
# 分离特征和目标
X = data.drop('强度', axis=1)
y = data['强度']

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 标准化特征
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 转换为PyTorch张量
X_train_tensor = torch.FloatTensor(X_train_scaled)
y_train_tensor = torch.FloatTensor(y_train.values).reshape(-1, 1)
X_test_tensor = torch.FloatTensor(X_test_scaled)
y_test_tensor = torch.FloatTensor(y_test.values).reshape(-1, 1)

## 4. 定义数据集和数据加载器

In [ ]:
class ConcreteDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_dataset = ConcreteDataset(X_train_tensor, y_train_tensor)
test_dataset = ConcreteDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

## 5. 定义多层感知机模型

我们将创建一个三层的多层感知机模型。

In [ ]:
class MLP(nn.Module):
    def __init__(self, input_dim):
        super(MLP, self).__init__()
        self.layer1 = nn.Linear(input_dim, 64)
        self.layer2 = nn.Linear(64, 32)
        self.layer3 = nn.Linear(32, 1)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.relu(self.layer2(x))
        x = self.layer3(x)
        return x

# 实例化模型
model = MLP(input_dim=X_train.shape[1])
print(model)

## 6. 定义损失函数和优化器

In [ ]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

## 7. 训练模型

In [ ]:
num_epochs = 100
train_losses = []
test_losses = []

for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    
    train_loss /= len(train_loader)
    train_losses.append(train_loss)
    
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            test_loss += loss.item()
    
    test_loss /= len(test_loader)
    test_losses.append(test_loss)
    
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}')

# 绘制损失曲线
plt.plot(train_losses, label='Train Loss')
plt.plot(test_losses, label='Test Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Test Loss')
plt.legend()
plt.show()

## 8. 评估模型

In [ ]:
model.eval()
predictions = []
actual = []

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        outputs = model(X_batch)
        predictions.extend(outputs.numpy().flatten())
        actual.extend(y_batch.numpy().flatten())

predictions = np.array(predictions)
actual = np.array(actual)

# 计算均方根误差
mse = np.mean((predictions - actual) ** 2)
rmse = np.sqrt(mse)
print(f'Root Mean Squared Error: {rmse:.2f} MPa')

# 绘制预测值与实际值的散点图
plt.figure(figsize=(10, 6))
plt.scatter(actual, predictions, alpha=0.5)
plt.plot([actual.min(), actual.max()], [actual.min(), actual.max()], 'r--', lw=2)
plt.xlabel('actual strength (MPa)')
plt.ylabel('predicted strength (MPa)')
plt.title('Actual vs. Predicted Strength')
plt.legend(['Pridiction & Actual', 'Perfect Prediction'])
plt.show()

## 9. 使用模型进行预测

In [ ]:
# 假设我们要预测一个新的混凝土配比的强度
new_concrete = np.array([
    [350, 180, 1000, 800, 10, 28]  # 水泥, 水, 粗骨料, 细骨料, 外加剂, 龄期
])

# 标准化新数据
new_concrete_scaled = scaler.transform(new_concrete)
new_concrete_tensor = torch.FloatTensor(new_concrete_scaled)

# 预测
model.eval()
with torch.no_grad():
    prediction = model(new_concrete_tensor)

print(f'预测的混凝土强度: {prediction.item():.2f} MPa')

# 解释预测结果
print("\n解释:")
print("这个预测结果是基于给定的混凝土配比和龄期得出的。")
print("在实际应用中,还需要考虑其他因素,如养护条件、环境温度等。")
print("这个模型可以帮助工程师快速评估混凝土配比的强度,并进行优化。")

## 10. 总结

在这个实践中，我们学习了如何使用PyTorch实现多层感知机(MLP)模型。我们以预测混凝土抗压强度为例，这是土木工程中的一个重要问题。
主要步骤包括:
- 准备数据
- 数据预处理
- 定义数据集和数据加载器
- 定义多层感知机模型
- 定义损失函数和优化器
- 训练模型
- 评估模型
- 使用模型进行预测

与线性回归模型相比，多层感知机模型可以更好地拟合非线性关系。因此，在处理复杂问题时，多层感知机模型通常比线性回归模型更有效。